In [1]:
import datetime as dt
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn import preprocessing

# Data Input, EDA & Preprocessing

## Individual Data Input

In [2]:
price_vol = pd.read_csv('Price & Volume BTC.csv')
tweets = pd.read_csv('Tweets.csv')
funding_rates_1600 = pd.read_csv('Funding Rates BTCUSDT 1600.csv')
funding_rates_0800 = pd.read_csv('Funding Rates BTCUSDT 0800.csv')
funding_rates_0000 = pd.read_csv('Funding Rates BTCUSDT 0000.csv')
google = pd.read_csv('Google.csv')
transactions = pd.read_csv('Transactions.csv')
unique_addresses = pd.read_csv('Unique Addresses.csv')
active_addresses = pd.read_csv('Active Addresses.csv')

## Data Compilation

In [3]:
# get daily ave value for twitter attributes
tweets = tweets.groupby(by="Time").mean()
tweets = tweets.reset_index()

df = pd.merge(price_vol, funding_rates_0000, on=['Time'])
df = pd.merge(df, funding_rates_0800, on=['Time'])
df = pd.merge(df, funding_rates_1600, on=['Time'])
df = pd.merge(df, google, on=['Time'])
df = pd.merge(df, active_addresses, on=['Time'])
df = pd.merge(df, unique_addresses, on=['Time'])
df = pd.merge(df, transactions, on=['Time'])
df = pd.merge(tweets, df, how="outer")
df.head()

,Time,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,Close,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,2020-03-12,5.0,2.0,0.0772,0.091,0.100,0.809,7933.2,4802.9,7960.4,4448.6,55248355.79,0.02%,0.01%,0.01%,22.83,6.20,909601,653156.0,349978.0
1,2020-03-17,98.0,96.0,0.4019,0.000,0.114,0.886,5028.8,5316.7,5532.0,4926.1,18269210.08,-0.09%,-0.02%,0.00%,10.21,3.39,765477,540902.0,278326.0
2,2020-03-20,125.0,95.0,0.4767,0.000,0.146,0.854,6162.2,6204.4,6901.7,5661.9,37752569.82,-0.03%,-0.02%,0.00%,10.04,3.56,734614,564748.0,284594.0
3,2020-03-30,3.0,0.0,0.0803,0.061,0.071,0.869,5881.3,6394.9,6600.0,5860.0,17904828.40,-0.03%,-0.06%,-0.03%,5.92,3.63,770915,535787.0,276828.0
4,2020-03-31,186.0,142.0,0.6597,0.000,0.278,0.722,6395.0,6409.7,6520.0,6288.0,13159710.94,-0.01%,-0.01%,-0.02%,5.45,3.54,816889,546702.0,290227.0


In [4]:
print("Type of variables: ", "\n", df.dtypes)

Type of variables:  
 Time                        object
Favorites                  float64
Retweets                   float64
Compound                   float64
Negative                   float64
Positive                   float64
Neutral                    float64
Open                       float64
Close                      float64
High                       float64
Low                        float64
Volume                     float64
Funding Rate 0000           object
Funding Rate 0800           object
Funding Rate 1600           object
Bitcoin_SVI                float64
Cryptocurrency_SVI         float64
No. of Active Addresses      int64
No. of Unique Addresses    float64
Transactions               float64
dtype: object


# EDA & Preprocessing

In [5]:
# Select features, the target output is'Close'
feature_pool = ['Favorites','Retweets','Compound','Negative','Positive',\
                'Neutral','Open', 'High','Low',\
                'Volume','Funding Rate 0000','Funding Rate 0800','Funding Rate 1600','Bitcoin_SVI',\
                'Cryptocurrency_SVI','No. of Active Addresses','No. of Unique Addresses','Transactions']

output = 'Close'

## Data Type Transformation

In [6]:
# Transform object type of "Funding Rate 0000", "Funding Rate 0800" and " Funding Rate 1600" to numerical type
df['Funding Rate 0000'] = df['Funding Rate 0000'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 0800'] = df['Funding Rate 0800'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 1600'] = df['Funding Rate 1600'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))

## Train-Test Split

In [7]:
X, y = df.drop(output, axis=1), df[output]

# Split the data: train set: test set using = 70/30. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
print(y_train.sum()/y_train.count(),y_test.sum()/y_test.count())

31027.64285714286 34212.278995433786


## Handle Missing Data

In [8]:
# Count missing value
print(df.isna().sum())

# check the consistency of columns with missing value between train set and the whole set
print(df[feature_pool].isnull().any().tolist()==X_train.isnull().any().tolist())
print(df[output].isnull().any().tolist()==y_train.isnull().any().tolist())

Time                         0
Favorites                  458
Retweets                   458
Compound                   458
Negative                   458
Positive                   458
Neutral                    458
Open                         0
Close                        0
High                         0
Low                          0
Volume                       0
Funding Rate 0000            0
Funding Rate 0800            0
Funding Rate 1600            0
Bitcoin_SVI                  0
Cryptocurrency_SVI           0
No. of Active Addresses      0
No. of Unique Addresses      2
Transactions                 0
dtype: int64
False
True


### Monthly Sentiment Analysis

In [9]:
tweets.insert(1, "Month", "NaN")

tweets['Month'] = tweets['Time'].str[:7]
tweets.head(10)

,Time,Month,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,2020-03-12,2020-03,5.0,2.0,0.0772,0.091,0.100,0.809
1,2020-03-17,2020-03,98.0,96.0,0.4019,0.000,0.114,0.886
2,2020-03-20,2020-03,125.0,95.0,0.4767,0.000,0.146,0.854
3,2020-03-30,2020-03,3.0,0.0,0.0803,0.061,0.071,0.869
4,2020-03-31,2020-03,186.0,142.0,0.6597,0.000,0.278,0.722
5,2020-04-01,2020-04,171.0,120.0,0.5267,0.061,0.184,0.754
6,2020-04-16,2020-04,419.0,94.0,0.0000,0.000,0.000,1.000
7,2020-04-22,2020-04,73.0,18.0,0.3182,0.000,0.084,0.916
8,2020-04-30,2020-04,994.0,153.0,0.0000,0.000,0.000,1.000
9,2020-05-09,2020-05,35.0,2.0,0.0000,0.000,0.000,1.000


In [10]:
tweets = tweets.groupby(by="Month").mean()
tweets = tweets.reset_index()

In [11]:
X_train.insert(1, "Month", "NaN")
X_train['Month'] = X_train['Time'].str[:7]

X_test.insert(1, "Month", "NaN")
X_test['Month'] = X_test['Time'].str[:7]

In [12]:
df_X_train = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_train.iterrows():
    if (pd.isna(X_train.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_train.loc[i].at["Month"]]
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_train.loc[i].at["Open"], \
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': X_train.loc[i].at["Favorites"], 'Retweets': X_train.loc[i].at["Retweets"], 'Compound': X_train.loc[i].at["Compound"], \
                   'Negative': X_train.loc[i].at["Negative"], 'Positive': X_train.loc[i].at["Positive"], 'Neutral': X_train.loc[i].at["Neutral"], 'Open': X_train.loc[i].at["Open"],\
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)


In [13]:
df_X_test = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_test.iterrows():
    if (pd.isna(X_test.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_test.loc[i].at["Month"]]
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': X_test.loc[i].at["Favorites"], 'Retweets': X_test.loc[i].at["Retweets"], 'Compound': X_test.loc[i].at["Compound"], \
                   'Negative': X_test.loc[i].at["Negative"], 'Positive': X_test.loc[i].at["Positive"], 'Neutral': X_test.loc[i].at["Neutral"], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)


In [14]:
X_test = df_X_test
X_test = X_test.drop(['Time'], axis=1)
X_test = X_test.drop(['Month'], axis=1)
X_test

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,2672.116667,532.600000,0.161596,0.042489,0.096383,0.861117,66007.2,66663.4,62015.3,8.638662e+08,0.06,0.07,0.06,26.79,32.64,1139133,765389.0,320743.0
1,904.500000,113.500000,0.368450,0.000000,0.140000,0.860000,47291.7,47577.0,45700.0,3.266873e+08,0.01,0.01,0.01,13.98,34.09,948843,630932.0,234793.0
2,1893.148148,241.161111,0.417171,0.030372,0.158927,0.810756,56678.4,57990.0,48352.8,5.226733e+08,0.03,0.02,0.03,22.03,57.53,1245595,852846.0,300452.0
3,1429.833333,203.333333,0.314467,0.022333,0.130500,0.847167,50055.1,50210.1,45714.5,4.840640e+08,0.01,0.01,0.01,14.80,32.22,996713,690479.0,278211.0
4,1769.357143,184.857143,0.241271,0.002214,0.134714,0.863143,34240.7,34914.9,32810.0,1.157770e+08,0.17,0.04,0.03,21.71,41.82,1087755,756022.0,312313.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,303.750000,79.000000,0.227450,0.000000,0.065250,0.934750,9299.6,9339.5,9204.6,1.948124e+07,0.01,0.01,0.01,3.41,5.36,909653,638906.0,310227.0
215,1199.634921,155.000000,0.385409,0.007032,0.134857,0.858127,59863.4,63759.7,59815.0,4.305186e+08,0.09,0.08,0.09,26.42,43.37,1178027,839652.0,310213.0
216,4.000000,3.000000,-0.659700,0.252000,0.000000,0.748000,17802.8,18808.6,17747.4,4.032508e+07,0.05,0.04,0.04,19.15,7.43,1177776,772326.0,348026.0
217,83.400000,67.000000,0.339160,0.030400,0.141800,0.828000,8035.0,8175.2,7636.3,2.070482e+07,0.04,0.01,0.01,16.05,5.16,809850,606002.0,348211.0


In [15]:
X_train = df_X_train
X_train = X_train.drop(['Time'], axis=1)
X_train = X_train.drop(['Month'], axis=1)
X_train

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,6836.250000,1147.250000,0.260025,0.044500,0.123000,0.832500,56877.8,58331.0,55600.0,4.170301e+08,0.01,0.01,0.01,16.28,38.55,976682,713194.0,284719.0
1,353.514286,61.028571,0.006329,0.062714,0.070200,0.867114,43174.8,44966.0,42796.9,4.448582e+08,-0.01,0.01,0.01,17.67,29.34,1065354,722863.0,280634.0
2,414.250000,96.250000,0.211225,0.015250,0.067000,0.917500,7359.7,7371.6,6851.0,1.512158e+07,0.01,0.01,-0.01,6.91,5.10,730620,544185.0,286706.0
3,74.400000,21.400000,0.070720,0.063800,0.078800,0.857400,57420.6,57517.8,44160.0,3.305140e+08,0.13,0.12,0.09,33.74,24.61,1094544,761941.0,285383.0
4,62.000000,13.000000,0.809100,0.000000,0.455000,0.545000,38179.7,38366.0,37350.0,2.052313e+08,0.00,0.01,0.01,12.96,27.88,780477,553241.0,207173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,2683.000000,176.000000,0.429550,0.000000,0.096000,0.904000,35064.9,36663.0,34600.8,3.624227e+08,0.01,0.01,0.00,18.00,30.36,764279,554355.0,203986.0
507,1769.357143,184.857143,0.241271,0.002214,0.134714,0.863143,35469.6,35597.5,30064.7,2.245245e+08,0.02,0.04,0.05,20.81,12.01,1059467,736588.0,290754.0
508,303.750000,79.000000,0.227450,0.000000,0.065250,0.934750,9199.0,9224.9,9046.4,1.695723e+07,0.01,0.01,0.01,5.58,5.56,966383,709099.0,345849.0
509,547.000000,71.666667,0.559867,0.000000,0.186333,0.813667,48124.3,48520.0,47012.4,2.158584e+08,0.01,0.01,0.01,10.26,22.29,1021047,673139.0,271223.0


### Unique Addresses

In [16]:
KNN_miss_filling = KNNImputer(n_neighbors=2).fit(X_train)
X_train = pd.DataFrame(KNN_miss_filling.transform(X_train), columns=feature_pool)
X_test = pd.DataFrame(KNN_miss_filling.transform(X_test), columns=feature_pool)

In [17]:
print(X_train.isna().sum())
print(X_test.isna().sum())

Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
No. of Unique Addresses    0
Transactions               0
dtype: int64
Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
N

In [18]:
X_test.to_csv('X_test.csv', index=False)

In [19]:
X_train.to_csv('X_train.csv', index=False)

In [20]:
y_train.to_csv('Y_train.csv', index=False)

In [21]:
y_test.to_csv('Y_test.csv', index=False)